<a href="https://colab.research.google.com/github/jjangmo91/GWLRS/blob/main/GWLRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# 라이브러리 설치 및 Import
# Colab 환경에서 최초 실행 시 아래 주석을 해제하여 설치하세요.
# !pip install earthengine-api
# !pip install geemap

import ee
import geemap

# GEE 인증 및 초기화
ee.Authenticate()
ee.Initialize(project='ee-jjangmo91')

# 관심 영역 정의 (여의도)
aoi = ee.Geometry.Polygon(
    [[[126.920, 37.532],
      [126.920, 37.520],
      [126.950, 37.520],
      [126.950, 37.532]]],
)

# Landsat 8 이미지 컬렉션 불러오기
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')

# 첫 번째 시기 이미지(예: 2021년 6월)
image1 = (landsat_collection
          .filterBounds(aoi)
          .filterDate('2021-06-01', '2021-06-30') # 한달치 이미지 불러오기
          .sort('CLOUD_COVER') # 구름량 적은 이미지부터 순서대로 컬렉션 정렬
          .first()) # 가장 구름이 적은 1장의 이미지 선

# 두 번째 시기 이미지(예: 2021년 7월)
image2 = (landsat_collection
          .filterBounds(aoi)
          .filterDate('2021-07-01', '2021-07-31')
          .sort('CLOUD_COVER')
          .first())

# 시각화
Map = geemap.Map(center=[37.526, 126.935], zoom=13)
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # RGB
    'min': 0,
    'max': 0.3
}

# 레이어 추가
Map.addLayer(image1, vis_params, '2021년 6월')
Map.addLayer(image2, vis_params, '2021년 7월')
Map.addLayer(aoi, {}, 'AOI')
Map

Map(center=[37.526, 126.935], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [13]:
# 이미지 로컬 다운로드(TIFF)
from google.colab import files

# 파일명 지정
filename1 = 'landsat_2021_06_rgb.tif'
filename2 = 'landsat_2021_07_rgb.tif'

# 첫 번째 시기 이미지 내보내기 & 다운로드
geemap.ee_export_image(
    ee_object=image1.visualize(
        bands=['B4', 'B3', 'B2'],
        min=0,   # 최소 반사율
        max=0.3  # 최대 반사율
    ),
    filename=filename1,
    scale=30,
    region=aoi,
    file_per_band=False
)
files.download(filename1)

# 두 번째 시기 이미지 내보내기 & 다운로드
geemap.ee_export_image(
    ee_object=image2.visualize(
        bands=['B4', 'B3', 'B2'],
        min=0,   # 최소 반사율
        max=0.3  # 최대 반사율
    ),
    filename=filename2,
    scale=30,
    region=aoi,
    file_per_band=False
)
files.download(filename2)

print("이미지 내보내기 완료!")

Generating URL ...
Please wait ...
Data downloaded to /content/landsat_2021_06_rgb.tif


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Generating URL ...
Please wait ...
Data downloaded to /content/landsat_2021_07_rgb.tif


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

이미지 내보내기 완료!
